<a href="https://colab.research.google.com/github/mdorodch/ITSC3155-Demos/blob/master/Hour8_endToend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install trl #transformer reinforcement learning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
# 0. imports
import torch
from transformers import GPT2Tokenizer

from trl import AutoModelForCausalLMWithValueHead
from trl import PPOConfig, PPOTrainer

In [ ]:
# 1. load a pretrained model
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 2. initialize trainer
arguments = {"batch_size": 1}
config = PPOConfig(**arguments)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:239: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
# 3. encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

In [17]:
# 4. generate model response
generation_args = {"min_length": -1,  # don't ignore the EOS token
                    "top_k": 0.0,       # no top-k sampling
                    "top_p": 1.0,       # no nucleus sampling
                    "do_sample": True,
                    "pad_token_id": tokenizer.eos_token_id,# most decoder models don't have a padding token - use EOS token instead
                    "max_new_tokens": 32,}

response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_args)
response_txt = tokenizer.decode(response_tensor[0])
print(response_txt)

 Bandstand Shopping Center in Jefferson Bank. The store is known for being clean, but to visit my parents to order a car for Christmas, it must have


In [16]:
# 5. define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0, device=model.pretrained_model.device)]
print(reward)

# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
print(train_stats)


[tensor(1., device='cuda:0')]
{'objective/kl': -0.5133419036865234, 'objective/kl_dist': -0.5133419036865234, 'objective/logprobs': array([[ -8.238598  ,  -2.4991455 ,  -2.7534556 ,  -0.47965628,
         -1.4376003 ,  -8.932602  ,  -1.1640692 ,  -8.509733  ,
         -2.8266852 ,  -0.564463  ,  -1.5803381 ,  -3.3349586 ,
         -1.6680019 ,  -2.7944982 , -11.560055  ,  -7.9804735 ,
         -1.5951002 , -10.877927  ,  -3.681796  ,  -0.7275351 ,
         -4.0953655 ,  -7.988533  ,  -1.9724536 ,  -4.5022035 ,
         -2.8690872 ,  -2.8575218 ,  -2.3017683 ,  -5.8882055 ,
         -3.3472033 ,  -3.0218725 ,  -1.018899  ,  -5.2035146 ,
         -4.6867523 ,  -1.9218835 ,  -1.0306273 ,  -5.937719  ,
         -0.8348831 , -12.9496975 ]], dtype=float32), 'objective/ref_logprobs': array([[ -8.146135  ,  -2.249075  ,  -2.6608307 ,  -0.63042396,
         -1.6609863 ,  -9.227759  ,  -1.4445829 ,  -8.057437  ,
         -2.0270805 ,  -0.3717848 ,  -1.4963539 ,  -3.5336192 ,
         -1.3375975 